## School performance dataset

In [1]:
import pandas as pd


In [13]:
performance_df = pd.read_csv("../transformed_data/school_performance_per_municipality.csv")

In [25]:
performance_df

,Ano,Região,UF,Código do Município,Nome do Município,Localização,Dependência Administrativa,Aprovação,Reprovação,Abandono
0,2008,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Publico,--,--,--
1,2008,Norte,RO,1100023,ARIQUEMES,Rural,Publico,--,--,--
2,2008,Norte,RO,1100049,CACOAL,Rural,Publico,--,--,--
3,2008,Norte,RO,1100056,CEREJEIRAS,Rural,Publico,--,--,--
4,2008,Norte,RO,1100064,COLORADO DO OESTE,Rural,Publico,77.7,22,0.3
...,...,...,...,...,...,...,...,...,...,...
150498,2022,Centro-Oeste,GO,5222005,Vianópolis,Urbana,Pública,99.2,0.8,0.0
150499,2022,Centro-Oeste,GO,5222054,Vicentinópolis,Urbana,Pública,99.3,0.7,0.0
150500,2022,Centro-Oeste,GO,5222203,Vila Boa,Urbana,Pública,100.0,0.0,0.0
150501,2022,Centro-Oeste,GO,5222302,Vila Propício,Urbana,Pública,90.5,5.2,4.3


In [40]:
performance_list = list(performance_df['Código do Município'].unique())

In [33]:
def check_equal_lists(list1, list2):
    return sorted(list1) == sorted(list2)

## School infrastructure dataset

In [15]:
infrastructure_df = pd.read_csv("../transformed_data/school_infrastructure_per_municipality.csv")

In [27]:
infrastructure_df

,Região Geográfica,Unidade da Federação,Município,Código do Município,Rede,Matrículas,Ano,Docentes,Estabelecimentos,Turmas
0,Centro-Oeste,Goiás,Abadia de Goiás,5200050.0,Rural,0,2009,0,0.0,0
1,Centro-Oeste,Goiás,Abadia de Goiás,5200050.0,Urbana,302,2009,19,1.0,12
2,Sudeste,Minas Gerais,Abadia dos Dourados,3100104.0,Rural,0,2009,0,0.0,0
3,Sudeste,Minas Gerais,Abadia dos Dourados,3100104.0,Urbana,211,2009,16,1.0,7
4,Centro-Oeste,Goiás,Abadiânia,5200100.0,Rural,0,2009,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
144675,Nordeste,Bahia,Érico Cardoso,2900504.0,Urbana,475,2010,23,1.0,18
144676,Norte,Pará,Óbidos,1505106.0,Rural,0,2010,0,0.0,0
144677,Norte,Pará,Óbidos,1505106.0,Urbana,"2,405",2010,64,1.0,85
144678,Sudeste,São Paulo,Óleo,3533809.0,Rural,0,2010,0,0.0,0


In [41]:
infra_list = list(infrastructure_df['Código do Município'].unique())

In [42]:
check_equal_lists(performance_list, infra_list)

True

In [44]:
performance_name_list = list(performance_df['Nome do Município'].unique())
infra_name_list = list(infrastructure_df['Município'].unique())
check_equal_lists(performance_name_list, infra_name_list)

False

(5308, 10684)

## Funding Dataset

In [46]:
funding_df = pd.read_csv("../transformed_data/funding_per_municipality.csv")

In [47]:
funding_df

,Município,Ano,UF,Código IBGE,Valor Consolidado
0,Abadia de Goiás,2007,GO,5200050,1261780.19
1,Abadia de Goiás,2008,GO,5200050,1755502.19
2,Abadia de Goiás,2009,GO,5200050,1880883.04
3,Abadia de Goiás,2010,GO,5200050,2324131.10
4,Abadia de Goiás,2011,GO,5200050,2850424.86
...,...,...,...,...,...
94614,Óleo,2019,SP,3533809,1565105.07
94615,Óleo,2020,SP,3533809,1425284.00
94616,Óleo,2021,SP,3533809,1691621.32
94617,Óleo,2022,SP,3533809,1729157.99


In [48]:
funding_list = list(funding_df['Código IBGE'].unique())

In [49]:
check_equal_lists(performance_list, funding_list)

False

In [50]:
len(performance_list), len(funding_list)

(5570, 5569)

In [51]:
def is_subset(list1, list2):
    return set(list1).issubset(set(list2))


In [52]:
is_subset(funding_list, performance_list)

True

## Population dataset

In [53]:
population_df = pd.read_csv("../transformed_data/population_per_municipality.csv")
population_df

,Código do Município,Nome do Município,UF,Ano,Population
0,1200013,Acrelândia,AC,2008,11987
1,1200013,Acrelândia,AC,2009,12241.0
2,1200013,Acrelândia,AC,2010,12510.0
3,1200013,Acrelândia,AC,2011,12779.0
4,1200013,Acrelândia,AC,2012,13011
...,...,...,...,...,...
83545,1722107,Xambioá,TO,2018,11561
83546,1722107,Xambioá,TO,2019,11540
83547,1722107,Xambioá,TO,2020,11520
83548,1722107,Xambioá,TO,2021,11500


In [55]:
population_list = list(population_df['Código do Município'].unique())

In [56]:
check_equal_lists(performance_list, population_list)

True